In [3]:
# Cell 1: Setup & imports
import os
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from scipy import stats

# Directories
BASE = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path().resolve().parents[0]

CLEANED = BASE / "data" / "cleaned"
FIG_DIR = BASE / "data" / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

# plotly default template
px.defaults.template = "plotly_white"

print("Using cleaned data from:", CLEANED)


Using cleaned data from: D:\marketing_dashboard_project\data\cleaned


In [4]:
# Cell 2: Load data
def read_prefixed(name):
    p_parquet = CLEANED / f"{name}.parquet"
    p_csv = CLEANED / f"{name}.csv"
    if p_parquet.exists():
        return pd.read_parquet(p_parquet)
    elif p_csv.exists():
        return pd.read_csv(p_csv, parse_dates=["date"])
    else:
        print(f"[WARN] {name} not found in {CLEANED}")
        return None

daily_total = read_prefixed("metrics_daily_total")
daily_channel = read_prefixed("metrics_daily_channel")
campaign = read_prefixed("campaign_metrics")
state = read_prefixed("state_metrics")
top_spend = read_prefixed("top_campaign_by_spend")
top_roas = read_prefixed("top_campaign_by_roas")

# Basic checks
print("daily_total:", None if daily_total is None else daily_total.shape)
print("daily_channel:", None if daily_channel is None else daily_channel.shape)
print("campaign:", None if campaign is None else campaign.shape)
print("state:", None if state is None else state.shape)

# Ensure date columns are datetime.date where appropriate
if daily_total is not None and 'date' in daily_total.columns:
    daily_total['date'] = pd.to_datetime(daily_total['date']).dt.date
if daily_channel is not None and 'date' in daily_channel.columns:
    daily_channel['date'] = pd.to_datetime(daily_channel['date']).dt.date


daily_total: (120, 22)
daily_channel: (360, 12)
campaign: (30, 12)
state: (2, 9)


In [5]:
# Cell 3: Time series (7-day MA)
print("## Time-series: Spend vs Total Revenue (7-day MA)")

if daily_total is None:
    print("metrics_daily_total missing. Run scripts 02/03 first.")
else:
    df = daily_total.sort_values("date").copy()
    df['spend_7d'] = df['spend'].rolling(7, min_periods=1).mean()
    df['total_revenue_7d'] = df['total_revenue'].rolling(7, min_periods=1).mean()

    fig = px.line(df, x='date', y=['spend_7d','total_revenue_7d'],
                  labels={'value':'USD','variable':'Metric'},
                  title='7-day MA: Spend vs Total Revenue')
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    fig.show()

    # save PNG (requires kaleido)
    try:
        fig.write_image(str(FIG_DIR / "timeseries_spend_revenue.png"), width=1200, height=450)
        print("Saved timeseries to:", FIG_DIR / "timeseries_spend_revenue.png")
    except Exception as e:
        print("Could not save PNG (kaleido missing?) - interactive display only.", e)


## Time-series: Spend vs Total Revenue (7-day MA)


Saved timeseries to: D:\marketing_dashboard_project\data\figures\timeseries_spend_revenue.png


In [6]:
# Cell 4: Channel performance
print("## Channel performance")

if daily_channel is None:
    print("metrics_daily_channel missing.")
else:
    ch = daily_channel.groupby('channel', as_index=False).agg({
        'spend':'sum','attributed_revenue':'sum','impression':'sum','clicks':'sum'
    })
    ch['roas'] = ch.apply(lambda r: (r['attributed_revenue']/r['spend']) if r['spend']>0 else np.nan, axis=1)
    ch = ch.sort_values('spend', ascending=False)

    fig = px.bar(ch, x='channel', y=['spend','attributed_revenue'], barmode='group',
                 title='Total Spend vs Attributed Revenue by Channel (period)')
    fig.show()
    try:
        fig.write_image(str(FIG_DIR / "channel_spend_revenue.png"), width=1000, height=450)
    except Exception:
        pass

    fig_roas = px.bar(ch.sort_values('roas', ascending=False), x='channel', y='roas',
                      title='Channel ROAS (attribution)')
    fig_roas.show()
    try:
        fig_roas.write_image(str(FIG_DIR / "channel_roas.png"), width=800, height=400)
    except Exception:
        pass

    display_cols = ['channel','spend','attributed_revenue','roas','impression','clicks']
    display_df = ch[display_cols].reset_index(drop=True)
    display_df.to_csv(FIG_DIR/"channel_breakdown.csv", index=False)
    display_df.head(20)


## Channel performance


In [7]:
# Cell 5: Campaign scatter (spend vs roas)
print("## Campaign scatter: Spend vs ROAS")

if campaign is None:
    print("campaign_metrics missing.")
else:
    camp = campaign.copy()
    # ensure numeric & avoid divide-by-zero
    camp['spend'] = pd.to_numeric(camp['spend'], errors='coerce').fillna(0.0)
    camp['attributed_revenue'] = pd.to_numeric(camp['attributed_revenue'], errors='coerce').fillna(0.0)
    camp['roas'] = camp['attributed_revenue'] / camp['spend'].replace(0, np.nan)
    camp['impression'] = pd.to_numeric(camp.get('impression', 0))
    camp['clicks'] = pd.to_numeric(camp.get('clicks', 0))

    fig = px.scatter(camp, x='spend', y='roas', size='attributed_revenue', color='channel',
                     hover_data=['campaign'], title='Campaign: Spend vs ROAS (size=attributed_revenue)', log_x=True)
    fig.update_layout(xaxis_title='Spend (USD)', yaxis_title='ROAS (Revenue/Spend)')
    fig.show()
    try:
        fig.write_image(str(FIG_DIR/"campaign_scatter.png"), width=1000, height=600)
    except Exception:
        pass

    # Top & bottom tables
    top_roas = camp.sort_values('roas', ascending=False).head(10)
    worst_roas = camp.sort_values('roas', ascending=True).head(10)
    top_roas.to_csv(FIG_DIR/"top_campaigns_by_roas.csv", index=False)
    worst_roas.to_csv(FIG_DIR/"worst_campaigns_by_roas.csv", index=False)
    top_roas[['channel','campaign','spend','attributed_revenue','roas']].head(10)


## Campaign scatter: Spend vs ROAS


In [8]:
# Cell 6: Funnel
print("## Funnel: Impressions -> Clicks -> Orders")

if daily_channel is None or daily_total is None:
    print("Need both daily_channel and daily_total to build funnel.")
else:
    total_impr = int(daily_channel['impression'].sum())
    total_clicks = int(daily_channel['clicks'].sum())
    total_orders = int(daily_total['orders'].sum())

    funnel_df = pd.DataFrame({
        'stage': ['Impressions', 'Clicks', 'Orders'],
        'value': [total_impr, total_clicks, total_orders]
    })
    fig = px.funnel(funnel_df, x='value', y='stage', title='Overall Funnel (period)')
    fig.show()
    try:
        fig.write_image(str(FIG_DIR/"funnel.png"), width=800, height=400)
    except Exception:
        pass

    # channel CTR table
    ch_ctr = daily_channel.groupby('channel', as_index=False).agg({'impression':'sum','clicks':'sum'})
    ch_ctr['ctr'] = ch_ctr['clicks'] / ch_ctr['impression'].replace(0, np.nan)
    ch_ctr.to_csv(FIG_DIR/"channel_ctr.csv", index=False)
    ch_ctr


## Funnel: Impressions -> Clicks -> Orders


In [9]:
# Cell 7: State-level ROAS (choropleth)
print("## State-level ROAS")

if state is None:
    print("state_metrics not available.")
else:
    st_df = state.copy()
    st_df['spend'] = pd.to_numeric(st_df.get('spend', 0), errors='coerce').fillna(0)
    st_df['attributed_revenue'] = pd.to_numeric(st_df.get('attributed_revenue', 0), errors='coerce').fillna(0)
    st_df['roas'] = st_df['attributed_revenue'] / st_df['spend'].replace(0, np.nan)
    # If state codes are full names, consider mapping to USPS codes. This assumes 2-letter codes.
    if st_df['state'].str.len().max() > 3:
        print("State column seems long (maybe full names) - using bar chart instead.")
        fig = px.bar(st_df.sort_values('spend', ascending=False).head(20),
                     x='state', y=['spend','attributed_revenue'], barmode='group',
                     title='Top 20 states by spend')
        fig.show()
    else:
        fig = px.choropleth(st_df, locations='state', locationmode='USA-states', color='roas',
                            hover_name='state', hover_data=['spend','attributed_revenue'],
                            color_continuous_scale='Viridis', title='State-level ROAS')
        fig.show()
    try:
        fig.write_image(str(FIG_DIR/"state_roas.png"), width=1000, height=600)
    except Exception:
        pass


## State-level ROAS


In [10]:
# Cell 8: Anomaly detection
print("## Anomaly detection")

if daily_total is None:
    print("daily_total missing - cannot run anomaly detection.")
else:
    dt = daily_total.copy().sort_values('date')
    # z-score on spend
    dt['spend_z'] = np.abs(stats.zscore(dt['spend'].fillna(0)))
    z_anoms = dt[dt['spend_z'] > 3][['date','spend','spend_z']].reset_index(drop=True)
    print("Z-score anomalies (spend):")
    display(z_anoms)

    # IsolationForest on [spend, attributed_revenue, clicks]
    X = dt[['spend','attributed_revenue','clicks']].fillna(0).values
    iso = IsolationForest(contamination=0.02, random_state=42)
    iso.fit(X)
    dt['anomaly_score'] = iso.decision_function(X)
    dt['anomaly'] = iso.predict(X)  # -1 anomaly, 1 normal
    iso_anoms = dt[dt['anomaly'] == -1][['date','spend','attributed_revenue','clicks','anomaly_score']].reset_index(drop=True)
    print("IsolationForest anomalies:")
    display(iso_anoms.head(20))

    # Save anomaly lists
    z_anoms.to_csv(FIG_DIR/"anomalies_z_spend.csv", index=False)
    iso_anoms.to_csv(FIG_DIR/"anomalies_isoforest.csv", index=False)


## Anomaly detection
Z-score anomalies (spend):


,date,spend,spend_z


IsolationForest anomalies:


,date,spend,attributed_revenue,clicks,anomaly_score
0,2025-05-20,33217.76,92178.22,109106,-0.017126
1,2025-07-09,55520.98,157092.21,169286,-0.073725
2,2025-09-07,54280.65,154271.83,142422,-0.007197
